In [7]:
!pip install ensemble-boxes

'\nhttps://github.com/ZFTurbo/Weighted-Boxes-Fusion\n'

In [2]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO
from tqdm import tqdm
import os

In [3]:
# ensemble csv files

directory_path = ".\weightensemble"

# 모든 .csv 파일을 리스트로 저장
submission_files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith('.csv')]
submission_df = [pd.read_csv(file) for file in submission_files]

In [5]:
weights = [float(os.path.splitext(os.path.basename(file))[0]) / 10000 for file in submission_files]

# 결과 출력
print(weights)

[0.3064, 0.3107, 0.3565, 0.3703, 0.3718, 0.3968, 0.4226, 0.4263, 0.4331, 0.4385, 0.4442, 0.449, 0.4578, 0.5548, 0.5765, 0.5772, 0.5946, 0.6087, 0.6153, 0.6214, 0.6269, 0.6278, 0.6282, 0.6292, 0.6293, 0.6313, 0.632, 0.6326, 0.6328, 0.633, 0.6333, 0.6335, 0.6371, 0.6408, 0.6423, 0.6467, 0.6492, 0.6495, 0.6591, 0.6596, 0.6635, 0.6645, 0.6743, 0.6828]


In [6]:
print(f"Number of submission files: {len(submission_files)}")
print(f"Number of weights: {len(weights)}")

Number of submission files: 44
Number of weights: 44


In [7]:
image_ids = submission_df[0]['image_id'].tolist()
assert len(image_ids)==4871

In [8]:
annotation = r'C:\Users\z\OneDrive - Sejong University\바탕 화면\네이버 부스트캠프 제출용\네이버 부캠\프로젝트2\data\datatest\test.json' ### test.json 경로 넣기 ###
coco = COCO(annotation)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [9]:
prediction_strings = []
file_names = []
flag=0
for i, image_id in tqdm(enumerate(image_ids), total=len(image_ids)):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for idx,df in enumerate(submission_df):
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        # 바운딩 박스가 0개 혹은 1개인 경우 앙상블할 수 없으므로 패스합니다
        if len(predict_list)==0 or len(predict_list)==1:
            print(f"Warning: image_id {image_id} submission_df index {weights[idx]}has {len(predict_list)} bounding boxes. maybe ")
            flag=1
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        # Ensemble_boxes는 박스의 좌표가 0에서 1 사이일 것을 요구하므로 정규화해줍니다
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        
        # boxes, scores, labels = nms(boxes_list, scores_list, labels_list,iou_thr=iou_thr)
        # boxes, scores, labels = soft_nms(box_list, scores_list, labels_list, iou_thr=iou_thr)
        # boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list,iou_thr=iou_thr)
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list,weights,iou_thr=0.75)
        
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

#======= 앙상블 결과 출력할 경로 지정 ========#
submission.to_csv(rf"C:\Users\z\OneDrive - Sejong University\바탕 화면\네이버 부스트캠프 제출용\네이버 부캠\프로젝트2\a\ensemble용\datasetensembleweight45odderasecreate.csv", index=False)

#===========================================#

submission.head()

  0%|                                                                                         | 0/4871 [00:00<?, ?it/s]C:\Users\z\anaconda3\envs\reboot\lib\site-packages\ensemble_boxes\ensemble_boxes_wbf.py:63: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
  0%|                                                                                 | 1/4871 [00:00<31:39,  2.56it/s]C:\Users\z\anaconda3\envs\reboot\lib\site-packages\ensemble_boxes\ensemble_boxes_wbf.py:54: UserWarning: Y1 < 0 in box. Set it to 0.
  warnings.warn('Y1 < 0 in box. Set it to 0.')
C:\Users\z\anaconda3\envs\reboot\lib\site-packages\ensemble_boxes\ensemble_boxes_wbf.py:51: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
C:\Users\z\anaconda3\envs\rebo

  0%|▎                                                                               | 17/4871 [00:05<21:38,  3.74it/s]

  0%|▎                                                                               | 21/4871 [00:06<25:29,  3.17it/s]

  1%|▌                                                                               | 31/4871 [00:09<17:19,  4.65it/s]

  1%|▌                                                                               | 34/4871 [00:09<19:09,  4.21it/s]

...

 98%|████████████████████████████████████████████████████████████████████████████▏ | 4758/4871 [22:40<00:32,  3.52it/s]

 98%|████████████████████████████████████████████████████████████████████████████▋ | 4788/4871 [22:47<00:18,  4.39it/s]

 98%|████████████████████████████████████████████████████████████████████████████▋ | 4792/4871 [22:48<00:18,  4.27it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 4871/4871 [23:11<00:00,  3.50it/s]


,PredictionString,image_id
0,7 0.8553320230516918 215.8356475830078 51.4052...,test/0000.jpg
1,4 0.4214985749132868 343.8914794921875 250.175...,test/0001.jpg
2,1 0.5081446567975484 425.2687683105469 268.556...,test/0002.jpg
3,9 0.20673277010249447 145.97201538085938 262.8...,test/0003.jpg
4,0 0.32258728263323455 425.54443359375 408.1771...,test/0004.jpg


In [10]:
#confidence score 기준으로 새로 뽑음
#
# 박스개수 지우거나 score 로 자를때 사용
#

import numpy as np
import pandas as pd
df = submission
#df = pd.read_csv(rf"C:\Users\z\OneDrive - Sejong University\바탕 화면\네이버 부스트캠프 제출용\네이버 부캠\프로젝트2\a\ensemble용\datasetensembleweight45odderasecreate.csv")
image_id_column = df['image_id']
prediction_strings = df['PredictionString'].fillna('')
# prediction_strings는 기존 코드에서 생성된 결과입니다.

# 새로운 prediction_strings를 저장할 리스트
filtered_prediction_strings = []

# 바운딩 박스 개수를 저장할 리스트
bbox_counts_per_line = []

for prediction_string in prediction_strings:
    # 바운딩 박스 문자열을 공백으로 나누어 리스트로 변환
    boxes = prediction_string.strip().split()
    
    # 바운딩 박스가 있는 경우
    if len(boxes) > 0:
        # 각 바운딩 박스를 (label, score, x1, y1, x2, y2) 형태로 변환
        num_boxes = len(boxes) // 6
        box_data = []

        for i in range(num_boxes):
            label = int(boxes[i * 6])
            score = float(boxes[i * 6 + 1])
            x1 = float(boxes[i * 6 + 2])
            y1 = float(boxes[i * 6 + 3])
            x2 = float(boxes[i * 6 + 4])
            y2 = float(boxes[i * 6 + 5])
            
            # confidence score가 0.05 이상인 바운딩 박스만 추가
            #if score > 0.01:
            box_data.append((label, score, x1, y1, x2, y2))
        
        # confidence score로 정렬
        box_data.sort(key=lambda x: x[1], reverse=True)  # score 기준 내림차순 정렬
        
        # 상위 499개 바운딩 박스만 선택
        
        if len(box_data) > 400:
            box_data = box_data[:400]

        # 새로운 prediction_string 생성
        new_prediction_string = ''
        for box in box_data:
            new_prediction_string += f"{box[0]} {box[1]:.10f} {box[2]:.5f} {box[3]:.5f} {box[4]:.5f} {box[5]:.5f} "
            #new_prediction_string += f"{box[0]} {box[1]:.3f} {box[2]:.2f} {box[3]:.2f} {box[4]:.2f} {box[5]:.2f} "
        
        filtered_prediction_strings.append(new_prediction_string.strip())
        
        # 바운딩 박스 개수 저장
        bbox_counts_per_line.append(len(box_data))
    else:
        # 바운딩 박스가 없는 경우는 그대로 추가
        filtered_prediction_strings.append(prediction_string)
        bbox_counts_per_line.append(0)  # 바운딩 박스가 없으므로 0으로 저장

# 새로운 prediction_strings 업데이트

submission = pd.DataFrame()
submission['PredictionString'] = filtered_prediction_strings
submission['image_id'] = image_id_column

#======= 앙상블 결과 출력할 경로 지정 ========#
submission.to_csv(".\datasetensembleweight_10241608.csv", index=False)

# 각 줄마다 바운딩 박스 개수 출력df
total = 0
for i, count in enumerate(bbox_counts_per_line):
    print(f"Line {i + 1}: {count} bounding boxes")
    total += count
print(total)


Line 1: 400 bounding boxes
Line 2: 400 bounding boxes
Line 3: 400 bounding boxes
Line 4: 400 bounding boxes
Line 5: 400 bounding boxes
Line 6: 400 bounding boxes
Line 7: 400 bounding boxes
Line 8: 400 bounding boxes
Line 9: 400 bounding boxes
Line 10: 400 bounding boxes
Line 11: 400 bounding boxes
Line 12: 400 bounding boxes
Line 13: 400 bounding boxes
Line 14: 400 bounding boxes
Line 15: 400 bounding boxes
Line 16: 400 bounding boxes
Line 17: 400 bounding boxes
Line 18: 400 bounding boxes
Line 19: 400 bounding boxes
Line 20: 400 bounding boxes
Line 21: 400 bounding boxes
Line 22: 400 bounding boxes
Line 23: 400 bounding boxes
Line 24: 400 bounding boxes
Line 25: 400 bounding boxes
Line 26: 400 bounding boxes
Line 27: 400 bounding boxes
Line 28: 400 bounding boxes
Line 29: 400 bounding boxes
Line 30: 400 bounding boxes
Line 31: 400 bounding boxes
Line 32: 400 bounding boxes
Line 33: 400 bounding boxes
Line 34: 400 bounding boxes
Line 35: 400 bounding boxes
Line 36: 400 bounding boxes
L